# Preprocessing

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 117.6 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import torch
import os
import random
import numpy as np
import random
import torch
import pandas as pd

torch.cuda.is_available()

True

In [3]:
# define colors for error messages style

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [4]:
# set seed

seed = 99
random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

no_deprecation_warning=True

In [5]:
""" LOAD FROM MULTIPLE FILES

model_name = "roberta-base"         # 12-layer, 768-hidden, 12-heads, 125M parameters
max_length = 512                    # max token length

n_real = 3000         # max 16762
n_scigen = 1200       # max 4865
n_gpt_2 =  1200       # max 4599
n_chat_gpt = 1200     # max 4909
n_galactica = 1200    # max 3582

# load and shuffle the datasets
real = pd.read_csv("real_papers.csv").sample(frac = 1).reset_index(drop=True)
real['label'] = 0
scigen = pd.read_csv("scigen_papers.csv").sample(frac = 1).reset_index(drop=True)
scigen['label'] = 1
gpt_2 = pd.read_csv("gpt2_papers.csv").sample(frac = 1).reset_index(drop=True)
gpt_2['label'] =1
chat_gpt = pd.read_csv("chatgpt_papers.csv").sample(frac = 1).reset_index(drop=True)
chat_gpt['label'] = 1
galactica = pd.read_csv("galactica_papers.csv").sample(frac = 1).reset_index(drop=True)
galactica['label'] = 1
"""

"""LOAD FROM ONE FILE"""

model_name = "roberta-base"         # 12-layer, 768-hidden, 12-heads, 125M parameters
max_length = 512                    # max token length

data = pd.read_csv("classifier_input_restricted_train.csv").sample(frac = 1).reset_index(drop=True)

# uncomment to set a different ratio of real data in the training set
count = 5000
real = data[data["src"]=='real'].sample(frac = 1).reset_index(drop=True)                        # save real data, shuffle them and reset indexes
data = data.loc[data['src']!='real'].reset_index(drop=True)                                     # remore real data from the original dataset
data = data.append(real[:count], ignore_index=True).sample(frac = 1).reset_index(drop=True)     # reintroduce only the desired portion 

# uncomment to remove chatgpt data from the training set                                      
data = data.loc[data['src']!='chatgpt'].reset_index(drop=True)                                       
                            
data = data.drop(columns=['id','year','categories','src'], axis = 1)            # remove unuseful cols
data

,title,abstract,introduction,conclusion,label
0,Multigrid methods for saddle point problems: K...,We consider saddle point problems for a class ...,The ﬁrst system is a two-point system with a s...,The problem of saddle point problems for the d...,1
1,Review of Smart Meter Data Analytics: Applicat...,Smart meters are becoming increasingly popular...,"Data Science, Engineering, and Mathematics (DO...",Monitoring and Evaluation of Smart Metrics in ...,1
2,Euclid: Superluminous supernovae in the Deep S...,Superluminous supernovae (SLSNe) are a rare cl...,Supernovae are some of the most powerful explo...,The Euclid Deep Survey has provided new insigh...,1
3,The Effect of Autonomous Models on Algorithms,Scheme must work. Given the current status of...,"Many end-users would agree that, had it not b...",In this work we veriﬁed that the UNIVAC compu...,1
4,"Fairness, Integrity, and Privacy in a Scalable...",The proposed Federated Learning System (FLS) i...,"SIR-FAR) is a decentralized, distributed, and ...",NSF grant AST-11-091. The impact of the Higgs ...,1
...,...,...,...,...,...
12995,The gauge theory of dislocations: conservation...,We derive conservation and balance laws for th...,In the Lagrangian ﬁeld theory of elasticity th...,We have presented the equations of motion for ...,0
12996,Neutrino Oscillations With Two Sterile Neutrinos,This work estimates the probability of $\mu$ t...,Reviews of experimental data on neutrino oscil...,From Figure 1 we note that even with the small...,0
12997,Extended main sequence turnoffs in open cluste...,We investigate the morphology of the colour-ma...,Originally discovered in massive intermediate ...,In this study we present Gaia photometry of se...,0
12998,The X-ray spectral properties of the AGN popul...,We present here a detailed X-ray spectral anal...,Recent deep X-rays surveys carried out by XMM-...,We have analyzed the X-ray spectra correspondi...,0


In [6]:
# uncomment to add gpt-3 papers from previous year into the training dataset
gpt3 = pd.read_csv("gpt3curienlp2022_restricted_train.csv").sample(frac = 1).reset_index(drop=True)
gpt3 = gpt3.drop(columns=['split','src'], axis = 1)
data = data.append(gpt3, ignore_index=True)
data = data.sample(frac = 1).reset_index(drop=True)
data

'TRAIN OLD GPT-3\ngpt3 = pd.read_csv("gpt3curienlp2022_restricted_train.csv").sample(frac = 1).reset_index(drop=True)\ngpt3 = gpt3.drop(columns=[\'split\',\'src\'], axis = 1)\ndata = data.append(gpt3, ignore_index=True)\ndata = data.sample(frac = 1).reset_index(drop=True)\ndata\n'

# Train

In [7]:
def train_val_split(datasets, dimentions, portion):

    if len(datasets) != len(dimentions): 
        print(bcolors.FAIL + "Error: dataset len is different from dimensions len" + bcolors.ENDC)
    
    train_val = []
    for i in range(len(datasets)): 
        data = datasets[i].copy()
        for j in range(dimentions[i]):
            # create the prompt e.g. "Abstract: ...put here the content..." and match the label 0 (real) - 1 (fake)
            train_val.append({'text' : str(portion).capitalize() + ': ' + str(data[portion][j]),'y' : int(data['label'][j])})

    # shuffle the data
    random.shuffle(train_val)

    X = [a['text'] for a in train_val]
    y = [a['y'] for a in train_val]
    
    # split into train and validation sets
    X_train, X_val, y_train, y_val =  train_test_split(X,y, test_size = 0.2, random_state=seed, stratify=y)
   
    return X_train, X_val, y_train, y_val 

In [8]:
# function called by the train method to compute the desired pre-defined metrics 

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [9]:
# class which iteratively returns a model input string as a tensor 

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
def train_data(portion):

  # load model and tokenizer from huggingface.co
  tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")
  
  
  # load and split the data 
  #X_train, X_val, y_train, y_val = train_val_split([real, scigen,gpt_2,chat_gpt,galactica], [n_real,n_scigen,n_gpt_2,n_chat_gpt,n_galactica],str(portion))
  X_train, X_val, y_train, y_val = train_val_split([data],[len(data)],str(portion))


  # tokenize the data
  train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=max_length)
  valid_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=max_length)

  # convert tokenized data into a torch Dataset
  train_dataset = Dataset(train_encodings, y_train)
  valid_dataset = Dataset(valid_encodings, y_val)

  training_args = TrainingArguments(
    output_dir='./results' + '/' + str(portion),  # output directory
    num_train_epochs=1,              
    per_device_train_batch_size=16,     # batch size per device during training
    per_device_eval_batch_size=16,      # batch size for evaluation
    logging_dir='./logs',               # directory for storing logs
    weight_decay=0.001,
    warmup_steps=1000,                  # number of updates with a low learning rate
    load_best_model_at_end=True,        # load the best model when finished training (default metric is loss)
    logging_steps=1000,                 # log weights each logging_steps
    save_steps=1000,                    # save weights each save_steps
    evaluation_strategy="steps",        # evaluate each `logging_steps`
    learning_rate = 5e-6,                 
    fp16 = True,                        # binary floating-point number format that occupies 16 bits 
    #gradient_accumulation_steps = 4,
  )

  trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments which are defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
  )

  trainer.train()
  print(trainer.evaluate())

  model_path = "./models/" + str(portion) 
  model.save_pretrained(model_path)      # save the model
  tokenizer.save_pretrained(model_path)

In [ ]:
train_data('abstract')

In [ ]:
train_data('introduction')

In [ ]:
train_data('conclusion')

# Test

In [ ]:
model_path = "./models"
model_name = "roberta-base"
max_length = 512     
target_names = [0,1]

In [ ]:
def get_prediction(text,model,tokenizer):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

In [ ]:
def load_test_data(datasets, dimentions):

    if len(datasets) != len(dimentions): 
        print(bcolors.FAIL + "Error: dataset len is different from dimensions len" + bcolors.ENDC)

    test = []

    for i in range(len(datasets)):
      
        data = datasets[i].copy()
        for j in range(dimentions[i]):
          test.append({
                  'abstract' : str('abstract').capitalize() + ': ' + str(data['abstract'][j]),
                  'introduction' :  str('introduction').capitalize() + ': ' + str(data['introduction'][j]),
                  'conclusion': str('conclusion').capitalize() + ': ' + str(data['conclusion'][j]),
                  'y' : int(data['label'][j])
                  })

    random.shuffle(test)
    X_a = [a['abstract'] for a in test]
    X_i = [i['introduction'] for i in test]
    X_c = [c['conclusion'] for c in test]
    y = [l['y'] for l in test]

    return X_a,X_i,X_c,y

In [ ]:
def test_data(X_test, y_test, portion):

  test_data = []
  correct = 0
  false = 0
  TP = 0
  TN = 0
  FP = 0
  FN = 0

  # reload the model
  model = AutoModelForSequenceClassification.from_pretrained(model_path + '/' + str(portion), num_labels=2).to("cuda")
  # load the tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

  for i in range(len(X_test)):
    # get prediction
    output = get_prediction(X_test[i],model,tokenizer)
    test_data.append(output)

    if output == y_test[i] : 
      correct +=1
      if y_test[i] == 0: TN +=1
      else: TP +=1

    else: 
      false +=1
      if y_test[i] == 0: FP += 1
      else: FN +=1
      
    print('Got', correct, '/', correct+false, ' correct, and ', false, '/', correct+false, 'wrong.', ' Current correct', y_test[i])
  
  # print final statistics
  print('Got', correct, '/', correct+false, ' correct, and ', false, '/', correct+false, 'wrong')
  print('TP: ',TP," TN: ",TN," FP: ",FP," FN: ",FN)
  print("\nAccuracy\t{},\nF1\t\t{},\nPrecision\t{},\nRecall\t\t{}".format((TP+TN)/(TP+TN+FP+FN),2*TP/(2*TP+FN+FP),TP/(TP+FP),TP/(TP+FN)))
  return(test_data)

In [ ]:
"""TEST OLD REAL"""
data = pd.read_csv("realnlp2022_restricted_4000.csv").sample(frac = 1).reset_index(drop=True)
data.drop(columns=['conc_tokens','intro_tokens','abs_tokens','src'], axis = 1)
X_a,X_i,X_c,y_test = load_test_data([data], [len(data)])

In [ ]:
"""TEST OLD GPT-3"""
data = pd.read_csv("gpt3curienlp2022_restricted_test.csv").sample(frac = 1).reset_index(drop=True)
data.drop(columns=['split','src'], axis = 1)
X_a,X_i,X_c,y_test = load_test_data([data], [len(data)])

In [ ]:
"""STANDARD TEST"""
data = pd.read_csv("classifier_input_restricted_test.csv").sample(frac = 1).reset_index(drop=True)
data.drop(columns=['id','year','categories','src'], axis = 1)

X_a,X_i,X_c,y_test = load_test_data([data], [len(data)])

In [ ]:
output_a = test_data(X_a,y_test,'abstract')

In [ ]:
output_i = test_data(X_a,y_test,'introduction')

In [ ]:
output_c = test_data(X_a,y_test,'conclusion')

In [ ]:
# Use the mode of the outcomames as prediction 
x = [output_a[i] + output_c[i] + output_i[i] for i in range(len(output_a))]

for i in range(len(x)):
  if x[i] > 1 : x[i] = 1
  else: x[i] = 0

np.sum(output_a), np.sum(output_i), np.sum(output_c), np.sum(x), len(x), len(y_test)

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(x)):
  if y_test[i] == 1 and x[i] == 1: TP+=1
  elif y_test[i] == 0 and x[i] == 1: FP+=1
  elif y_test[i] == 1 and x[i] == 0: FN+=1
  elif y_test[i] == 0 and x[i] == 0: TN+=1
  
# print final statistics
print('TP: ',TP," TN: ",TN," FP: ",FP," FN: ",FN)
print("Real: {}\nFake: {}".format(np.sum(y_test),len(y_test) - np.sum(y_test)))
print("\nAccuracy\t{},\nF1\t\t{},\nPrecision\t{},\nRecall\t\t{}".format((TP+TN)/(TP+TN+FP+FN),2*TP/(2*TP+FN+FP),TP/(TP+FP),TP/(TP+FN)))

# test-cc

In [ ]:
!pip install datasets
from datasets import load_dataset, concatenate_datasets, Dataset

In [ ]:
def get_test_cc_df():
  dataset = load_dataset('tum-nlp/IDMGSP', "test-cc")
  dataset = dataset['test']
  dataset_df = pd.DataFrame.from_dict(dataset)
  return dataset_df

In [ ]:
dataset = load_dataset('tum-nlp/IDMGSP', "test-cc")
dataset = dataset['test']
dataset_df = pd.DataFrame.from_dict(dataset)
dataset_df

In [ ]:
def predict_based_on_sections(full_test_cc, model_path):
  paraphrased_sections = pd.Series(full_test_cc["paraphrased_sections"].unique()).append(pd.Series(["all"]))
  print(paraphrased_sections)
  results = pd.DataFrame(columns = ['paraphrased_sections', 'abstract_accuracy', 'introduction_accuracy', 'conclusion_accuracy', 'complete_accuracy'])
  results['paraphrased_sections'] = paraphrased_sections
  for section in paraphrased_sections:
    if section == "all":
      subset = dataset_df
    else:
      subset = dataset_df[dataset_df["paraphrased_sections"] == section]
    subset = subset[["abstract", "introduction", "conclusion", "label"]].reset_index(drop=True)
    print(f"predicting rows with paraphrased_sections={section}")
    acc_complete, acc_a, acc_i, acc_c = predict_with_all_models(model_path = model_path, data = subset)
    results.loc[results['paraphrased_sections'] == section, 'abstract_accuracy'] = acc_a
    results.loc[results['paraphrased_sections'] == section, 'introduction_accuracy'] = acc_i
    results.loc[results['paraphrased_sections'] == section, 'conclusion_accuracy'] = acc_c
    results.loc[results['paraphrased_sections'] == section, 'complete_accuracy'] = acc_complete
  return results

In [ ]:
predict_based_on_sections(dataset_df, "train")

In [ ]:
predict_based_on_sections(dataset_df, "train+gpt3")

In [ ]:
predict_based_on_sections(dataset_df, "train-cg")

# Explainability

### Shap

In [27]:
! pip install shap
import shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
def load_model_tokenizer(portion): 
  model_name = "roberta-base"
  model_path = "./models/" + str(portion)
  model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2).to("cuda")
  tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)
  return model,tokenizer

portion = 'abstract' # could be also 'introduction' or 'conclusion'
model,tokenizer = load_model_tokenizer(portion)

loading configuration file ./models/abstract/config.json
Model config RobertaConfig {
  "_name_or_path": "./models/abstract",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file ./models/abstract/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of Roberta

In [29]:
def get_shap_values(text):
    pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, top_k=None, truncation=True, max_length=max_length, function_to_apply = "softmax")
    explainer = shap.Explainer(pipe)
    return explainer(text)

In [ ]:
data = pd.read_csv("classifier_input_restricted_val.csv")
# data = data[data['src']=='real']['abstract'][:10]                         # adapt this lines to the samples chosen 
# data = data.drop(columns=['id','year','categories','src'], axis = 1)
data

In [ ]:
shap_values = get_shap_values(data)                 # get the exaplanation
shap.plots.text(shap_values[:,:,0])                 # plot the explanation -> change the zero in (shap_values[:,:,0]) according to the class under evaluation
shap.plots.bar(shap_values[:,:,0].mean(0))          # plot the summary -> change the zero in (shap_values[:,:,0]) according to the class under evaluation

## Lime

In [ ]:
! pip install lime

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer

In [ ]:
def lime_prediction(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda") # tokenize the data
    outputs = model(**inputs)                                                                                      # predict the fakeness
    probs = outputs[0].softmax(1)                                                                                   
    return probs.to('cpu').detach().numpy()

In [ ]:
class_names = {0: 'real', 1:'fake'}
explainer = LimeTextExplainer(class_names=class_names)                                     
exp = explainer.explain_instance(data, lime_prediction, num_samples = 3) 
# plot the results
exp.show_in_notebook(text=True)